In [3]:
import requests
from bs4 import BeautifulSoup

def scrape_daum_economy_news(url):
    """
    Scrapes the provided Daum Economy news URL to extract news article links and titles.
    
    Args:
        url (str): The URL of the Daum Economy news page.
    
    Returns:
        None: Prints the extracted links and titles.
    """
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        response.encoding = 'utf-8'
        
        print(f"URL: {url}")
        print(f"상태 코드: {response.status_code}")
        
        soup = BeautifulSoup(response.text, 'html.parser')
        news_items = soup.select('ul.list_newsheadline2 li')
        print(f"발견된 뉴스 항목: {len(news_items)}개\n")
        
        for i, li_tag in enumerate(news_items, 1):
            a_tag = li_tag.find('a')
            if not a_tag or 'href' not in a_tag.attrs:
                continue
                
            link = a_tag['href']
            strong_tag = li_tag.select_one('div.cont_thumb strong.tit_txt')
            
            if strong_tag:
                title = strong_tag.text.strip()
                print(f"{i}. {title}")
                print(f"   {link}\n")
    
    except requests.exceptions.RequestException as e:
        print(f"요청 오류: {e}")
    except Exception as e:
        print(f"예상치 못한 오류: {e}")

# 실행
url = 'https://news.daum.net/economy'
scrape_daum_economy_news(url)

URL: https://news.daum.net/economy
상태 코드: 200
발견된 뉴스 항목: 9개

1. ‘해수부 부산 시대’ 조선·해양플랜트까지 업무 넓혀야
   https://v.daum.net/v/20250722202007801

2. [이슈대담] “2040년 석탄화력발전소 폐쇄…신재생 에너지 전환”
   https://v.daum.net/v/20250722200357366

3. 경제통상 외교수장들, 대거 미국행... 관세 타결·한미정상회담 열릴까
   https://v.daum.net/v/20250722190900992

4. 퇴근하면 시작된다...유통업계 다시 불붙은 '6시 쇼핑' 전쟁
   https://v.daum.net/v/20250722185103483

5. “자율주행, 이동약자 지원하는 ‘공공성’에 초점 맞춰야”
   https://v.daum.net/v/20250722183143976

6. 꾸준히 강세 흐름 보이는 메이저 알트코인들 : 코리안 크립토 위클리 [INFCL 리서치]
   https://v.daum.net/v/20250722181057244

7. 세계시장 제패한 K뷰티·푸드…'첨단' 아니라며 R&D지원은 쥐꼬리
   https://v.daum.net/v/20250722175548581

8. 예금자보호 1억 시대…내 돈 어디에 맡길까?
   https://v.daum.net/v/20250722174746199

9. 25일 美서 '2+2 관세 담판'…베선트 "印尼는 수정안 5번 들고와"
   https://v.daum.net/v/20250722174703180



In [4]:
import requests
from bs4 import BeautifulSoup

# 섹션 딕셔너리
section_dict = {
    '기후/환경': 'climate',
    '사회': 'society',
    '경제': 'economy',
    '정치': 'politics',
    '국제': 'world',
    '문화': 'culture',
    '생활': 'life',
    'IT/과학': 'tech',
    '인물': 'people'
}

def get_news_html(url: str) -> str | None:
    """주어진 URL에서 HTML 콘텐츠를 가져와서 UTF-8 인코딩을 보장합니다."""
    try:
        response = requests.get(url, timeout=10)
        response.encoding = 'utf-8'
        response.raise_for_status()
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"URL '{url}' 요청 오류: {e}")
        return None

def parse_news_articles(html_content: str) -> list[dict]:
    """HTML 콘텐츠를 파싱하여 뉴스 기사 링크와 제목을 추출합니다."""
    soup = BeautifulSoup(html_content, 'html.parser')
    news_items = []

    for li_tag in soup.select('ul.list_newsheadline2 li'):
        a_tag = li_tag.find('a')
        strong_tag = li_tag.select_one('div.cont_thumb strong.tit_txt')
        
        if a_tag and strong_tag and 'href' in a_tag.attrs:
            news_items.append({
                'link': a_tag['href'],
                'title': strong_tag.text.strip()
            })
    
    return news_items

def print_news(section_name: str):
    """
    다음 뉴스에서 지정된 섹션의 뉴스 기사를 조회하고 출력합니다.
    
    Args:
        section_name (str): 뉴스 섹션의 한국어 이름 (예: '경제', '사회')
    """
    section_key = section_dict.get(section_name)
    if not section_key:
        print(f"'{section_name}'은(는) 유효한 섹션명이 아닙니다.")
        print(f"사용 가능한 섹션: {', '.join(section_dict.keys())}\n")
        return

    target_url = f'https://news.daum.net/{section_key}'
    print(f"======> {section_name} 뉴스 ({target_url}) <======")

    html = get_news_html(target_url)
    if not html:
        print(f"'{section_name}' 섹션 페이지를 가져올 수 없습니다.\n")
        return
    
    articles = parse_news_articles(html)
    if not articles:
        print(f"'{section_name}' 섹션에서 뉴스를 찾을 수 없습니다.\n")
        return
    
    print(f"발견된 뉴스: {len(articles)}개\n")
    for i, article in enumerate(articles, 1):
        print(f"{i}. {article['title']}")
        print(f"   {article['link']}\n")

# --- 함수 호출 예시 ---
print_news('기후/환경')
print_news('사회')
print_news('경제')
print_news('정치')
print_news('인물')
print_news('없는섹션')

======> 기후/환경 뉴스 (https://news.daum.net/climate) <======
발견된 뉴스: 9개

1. 기후변화로 여름 폭우 한 달 빨라진다…7월이 ‘신(新) 장마 재난’ 중심
   https://v.daum.net/v/20250722201755739

2. [인천시, 탄소중립도시연합 가입] 세계적 기후 선도, 글로벌 협력 주도
   https://v.daum.net/v/20250722195504173

3. 환경 장관 “환경부 직원 전기차 타는 비율 국민 평균 수준”
   https://v.daum.net/v/20250722194703016

4. 베이징 대기질 개선 이끈 베이징대 석좌교수 "탄소중립이 건강"
   https://v.daum.net/v/20250722192326445

5. '기후부' 신설 빨라지나…환경장관 "산업부, 그대로 존재 안해"
   https://v.daum.net/v/20250722184347309

6. “동물 보호 외치면서 ‘뱀가죽’ 지갑?” 이런 게 수두룩…놀라운 ‘반전’ 알고 보니 [지구, 뭐래?]
   https://v.daum.net/v/20250722184158229

7. "곰 249마리 여전히 철창에"…동물·환경단체, 환경부에 책임 촉구
   https://v.daum.net/v/20250722173141469

8. KBSI, 지구온난화 따라 달라지는 땅속 유기물질 비밀 밝혀
   https://v.daum.net/v/20250722172644300

9. 김성환 환경부 장관님, 그 약속 꼭 지켜주세요
   https://v.daum.net/v/20250722172100084

======> 사회 뉴스 (https://news.daum.net/society) <======
발견된 뉴스: 9개

1. 여자배구 ‘코리아인비테이셔널 진주’서 명예회복 나선다
   https://v.daum.net/v/20250722201931785

2. 경기도 내 지자체, 고향사랑기부제 민간플랫폼 꺼린다
  